In [2]:
import numpy as np
import pandas as pd

## Load dataset

In [3]:
train = pd.read_csv("train_bike.csv", parse_dates = ["datetime"])
print(train.shape)
train.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [4]:
test = pd.read_csv("test_bike.csv", parse_dates = ["datetime"])
print(test.shape)
test.head()

(6493, 9)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


## Score

In [18]:
feature_names = ["season", "holiday", "workingday", "weather","temp", "atemp", "humidity", "windspeed"]

X_train = train[feature_names]

print(X_train.shape)
X_train.head()

(10886, 8)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,1,0,0,1,9.84,14.395,81,0.0
1,1,0,0,1,9.02,13.635,80,0.0
2,1,0,0,1,9.02,13.635,80,0.0
3,1,0,0,1,9.84,14.395,75,0.0
4,1,0,0,1,9.84,14.395,75,0.0


In [6]:
X_test = test[feature_names]
print(X_test.shape)
X_test.head()

(6493, 8)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,1,0,1,1,10.66,11.365,56,26.0027
1,1,0,1,1,10.66,13.635,56,0.0000
2,1,0,1,1,10.66,13.635,56,0.0000
3,1,0,1,1,10.66,12.880,56,11.0014
4,1,0,1,1,10.66,12.880,56,11.0014


In [7]:
label_name = "count"

y_train = train[label_name]

print(y_train.shape)
y_train.head()

(10886,)


0    16
1    40
2    32
3    13
4     1
Name: count, dtype: int64

In [8]:
from sklearn.ensemble import RandomForestRegressor

seed = 37

model = RandomForestRegressor(random_state=seed)
model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=37, verbose=0, warm_start=False)

In [26]:
from sklearn.metrics import make_scorer
def rmsle(predict, actual):
    predict = np.array(predict)
    actual = np.array(actual)
    
    log_predict = np.log(predict + 1)
    log_actual = np.log(actual + 1)
    
    difference = log_predict-log_actual
    square_difference = difference**2
    mean_square_difference = square_difference.mean()
    
    score = np.sqrt(mean_square_difference)
    
    return score


In [27]:
from sklearn.cross_validation import cross_val_score

score = cross_val_score(model, X_train, y_train, cv=30, scoring=rmsle_score).mean()

print("Score = {0:.5f}".format(score))

Score = 1.38852


In [28]:
model.fit(X_train, y_train)

predictions = model.predict(X_test)


print(predictions.shape)
predictions[:3]

(6493,)


array([104.55,  38.85,  38.85])

In [29]:
submission = pd.read_csv("sampleSubmission.csv")

submission["count"] = predictions

print(predictions.shape)

submission.head()

(6493,)


,datetime,count
0,2011-01-20 00:00:00,104.55
1,2011-01-20 01:00:00,38.85
2,2011-01-20 02:00:00,38.85
3,2011-01-20 03:00:00,97.30
4,2011-01-20 04:00:00,97.30


In [30]:
submission.to_csv("top.csv", index=False)